In [159]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Set the seed for reproducibility
np.random.seed(42)
# Time steps
T = np.arange(0, 365)


def create_time_series(
    t: np.ndarray,
    seasonalities: list[tuple[int, int]] = [(10, 2), (5, 4)],
    trend_coef: float = 0.1,
    noise_level: float = 2.0,
    plot: bool = False
):
    """Create a time series with a linear trend and seasonalities.
    
    Args:
        * t: an array of the time index
        * seasonalities: each list entry describes amplitude and frequency of a seasonal component.
        * trend_coef: the linear trend coefficient.
        * the noise level
        * plot (bool): if True, plot the time series
    """
    trend = trend_coef * t
    noise = np.random.normal(0, noise_level, len(t))
    time_series = trend + noise
    for amplitude, frequency in seasonalities:
        component = amplitude * np.sin(frequency * np.pi * t / 365)
        time_series += component

    if plot:
        plt.figure(figsize=(12, 6))
        plt.plot(t, time_series, label='Artificial Time Series')
        plt.xlabel('Time')
    return time_series



ts_at_bottom_level = pd.DataFrame(
    index=pd.date_range(start="2023-04-01", periods=365)
)
for column in ["AA", "AB", "BA", "BB"]:
    seasonal = np.random.randint(1, 15, size=6)
    seasonalities = list(map(tuple, seasonal.reshape(-1, 2)))
    ts_at_bottom_level[column] = create_time_series(
        T, seasonalities=seasonalities, trend_coef=2 + np.random.randn()
    )

In [160]:
ts_at_bottom_level.head()

,AA,AB,BA,BB
2023-04-01,4.283317,1.466347,-1.951960,-2.519840
2023-04-02,2.976153,2.855435,2.900329,4.234249
2023-04-03,4.450667,11.626423,6.541842,10.600114
2023-04-04,7.636099,13.995461,8.669148,17.091873
2023-04-05,11.613973,15.093862,9.018746,29.430580


In [161]:
# We can represent the hierarchy as either an aggregate sum matrix or as a hierarchy dictionary.
S = np.array([
    [1, 1, 1, 1],
    [1, 1, 0, 0],
    [0, 0, 1, 1],
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])
hierarchy = {
    "AA": ["A"],
    "AB": ["A"],
    "BA": ["B"],
    "BB": ["B"],
    "A": ["total"],
    "B": ["total"]
}

In [162]:
time_series = ts_at_bottom_level.assign(
    A=ts_at_bottom_level["AA"] + ts_at_bottom_level["AB"],
    B=ts_at_bottom_level["BA"] + ts_at_bottom_level["BB"],
)
time_series["total"] = time_series["A"] + time_series["B"]

In [163]:
from darts import TimeSeries
from darts.models import CatBoostModel
from darts.utils.model_selection import train_test_split

# Load and preprocess the data
series = TimeSeries.from_dataframe(time_series).with_hierarchy(hierarchy)

# Split the data into train and test sets
train, test = train_test_split(series, test_size=0.2)

# Train the model
model = CatBoostModel(
    lags=12
)
model.fit(train)

# Forecast the test set
forecast = model.predict(len(test))

In [164]:
from darts.metrics import mae, mse, rmse
# Evaluate the model
mae_bottom = mae(test, forecast)
mse_bottom = mse(test, forecast)
rmse_bottom = rmse(test, forecast)
print(f'MAE: {mae_bottom:.4f}, MSE: {mse_bottom:.4f}, RMSE: {rmse_bottom:.4f}')

MAE: 153.7264, MSE: 44531.8967, RMSE: 175.3400


In [165]:
from darts.dataprocessing.transformers import MinTReconciliator

reconciliator = MinTReconciliator(method="wls_val")
reconciliator.fit(train)
reconciled_preds = reconciliator.transform(forecast)

In [166]:
mae_recon = mae(test, reconciled_preds)
mse_recon = mse(test, reconciled_preds)
rmse_recon = rmse(test, reconciled_preds)
print(f'MAE: {mae_recon:.4f}, MSE: {mse_recon:.4f}, RMSE: {rmse_recon:.4f}')

MAE: 153.7067, MSE: 44511.2303, RMSE: 175.3358


In [167]:
(mae_recon - mae_bottom) / mae_bottom * 100

-0.012833308850691297